**We’ll use aiohttp for asynchronous HTTP requests, BeautifulSoup for parsing HTML, and csv for saving the data.**

In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time
import csv

# Set up the Chrome WebDriver (replace the path with where you've stored your chromedriver)
driver = webdriver.Chrome()  # Update with your path


**We need a helper function to convert game time strings (e.g., Q2 10:00) into total elapsed minutes.**

In [40]:
def convert_game_time(game_time):
    try:
        # Split the time string into quarter and minutes
        quarter, time_left = game_time.split(" ")
        minutes, seconds = map(int, time_left.split(":"))
        
        # Calculate elapsed time
        quarter_num = int(quarter[1:])  # Extract quarter number (e.g., Q2 -> 2)
        elapsed_minutes = (quarter_num - 1) * 12 + (12 - minutes)
        
        return elapsed_minutes
    except Exception as e:
        print(f"Error parsing game time: {game_time} - {e}")
        return None


**Create an asynchronous function to scrape live NBA games.**

In [41]:
async def scrape_nba_live_games():
    url = "https://www.nba.com/games"
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    async with aiohttp.ClientSession() as session:
        async with session.get(url) as response:
            if response.status == 200:
                html = await response.text()
                print(html)  # Print the HTML response to ensure you're getting the page content
                soup = BeautifulSoup(html, 'html.parser')
                
                # Extract game containers
                games = soup.find_all('div', class_='GameCard_gc__container__')
                live_games = []
                
                for game in games:
                    try:
                        team1 = game.find('div', class_='GameCard_team1__name__').text.strip()
                        team2 = game.find('div', class_='GameCard_team2__name__').text.strip()
                        score = game.find('div', class_='GameCard_score__').text.strip()
                        status = game.find('div', class_='GameCard_status__').text.strip()
                        
                        # Extract and convert time if available
                        time_div = game.find('div', class_='GameCard_time__')
                        time_str = time_div.text.strip() if time_div else "Q1 12:00"
                        elapsed_time = convert_game_time(time_str)
                        
                        live_games.append({
                            'Team 1': team1,
                            'Team 2': team2,
                            'Score': score,
                            'Status': status,
                            'Time': elapsed_time
                        })
                    except AttributeError:
                        continue
                
                return live_games
            else:
                print(f"Failed to fetch webpage. HTTP Status: {response.status}")
                return []


**Write the function to save data into a CSV file, ensuring it works even if no games are occurring.**

In [42]:
async def main():
    live_games = await scrape_nba_live_games()
    
    # Ensure the CSV file is created even if no games are found
    csv_file = "C:/Users/Admin/OneDrive/Máy tính/Học tập/2024 - 2025/Kỳ I 2024 - 2025/Machine Learning/Assignment/live_games_with_time.csv"
    with open(csv_file, mode="w", newline="", encoding="utf-8") as file:
        writer = csv.DictWriter(file, fieldnames=["Team 1", "Team 2", "Score", "Status", "Time"])
        writer.writeheader()
        if live_games:
            writer.writerows(live_games)
            print(f"Data saved to {csv_file}")
        else:
            print("No games are currently occurring. Empty CSV file created.")

In [5]:
from nba_api.stats.endpoints import leaguegamefinder
import pandas as pd

# Specify the season (e.g., "2024-25" for the current season)
season_id = "2024-25"

# Fetch games for the specified season
games = leaguegamefinder.LeagueGameFinder(season=season_id)

# Get the data as a Pandas DataFrame
games_df = games.get_data_frames()[0]

# Display the first few rows of the DataFrame
print(games_df.head())

# Optionally, save to a CSV file
games_df.to_csv('nba_games_2024_25.csv', index=False)


TypeError: LeagueGameFinder.__init__() got an unexpected keyword argument 'season'